In [69]:
import numpy as np
from collections import defaultdict

In [90]:
class Component:
    def __init__(self, name : str, node1 : int, node2 : int, value : float, initial_value : float):
        self.name = name
        self.node1 = node1
        self.node2 = node2
        self.value = value
        self.initial_value = initial_value
    
    def __str__(self):
        return f"{self.name} is connected between node{self.node1} and node{self.node2}. Current value is {self.value}. Initial value is {self.initial_value}"

In [140]:
class Simulation:
    def __init__(self, time_step, num_iter, n, m, components):
        self.time_step = time_step
        self.num_iter = num_iter
        self.n = n
        self.m = m
        self.capacitors = components['C']
        self.inductors = components['I']
        self.voltage_sources = components['Vsrc']
        self.current_sources= components['Isrc']
        self.resistances = components['R']
    
    def construct_Gmat(self):
        G = np.zeros((self.n, self.n))
        
        for res in self.resistances:
            if res.node1 == 0:
                G[res.node2 - 1][res.node2 - 1] += (1/res.value)
            elif res.node2 == 0:
                G[res.node1 - 1][res.node1 - 1] += (1/res.value)
            else:
                G[res.node1 - 1][res.node2 - 1] -= (1/res.value)
                G[res.node2 - 1][res.node1 - 1] -= (1/res.value)
                G[res.node1 - 1][res.node1 - 1] += (1/res.value)
                G[res.node2 - 1][res.node2 - 1] += (1/res.value)
        
        # Applying capacitors stamp for backward euler
        for cap in self.capacitors:
            if cap.node1 == 0:
                G[cap.node2 - 1][cap.node2 - 1] += (cap.value/self.time_step)
            elif cap.node2 == 0:
                G[cap.node1 - 1][cap.node1 - 1] += (cap.value/self.time_step)
            else:
                G[cap.node1 - 1][cap.node2 - 1] -= (cap.value/self.time_step)
                G[cap.node2 - 1][cap.node1 - 1] -= (cap.value/self.time_step)
                G[cap.node1 - 1][cap.node1 - 1] += (cap.value/self.time_step)
                G[cap.node2 - 1][cap.node2 - 1] += (cap.value/self.time_step)
        
        return G
    
    def construct_Bmat(self):
        B = np.zeros((self.n, self.m + len(self.inductors)))
        for index, voltage_source in enumerate(self.voltage_sources):
            if voltage_source.node1 != 0: 
                B[voltage_source.node1 - 1][index] = 1

            if voltage_source.node2 != 0:
                B[voltage_source.node2 - 1][index] = -1
        
        for index, ind in enumerate(self.inductors):
            if ind.node1 != 0: 
                B[ind.node1 - 1][self.m + index] = 1
            if ind.node2 != 0:
                B[ind.node2 - 1][self.m + index] = -1
        
        return B
    
    def construct_Cmat(self, B_matrix):
        return np.transpose(B_matrix)
    
    def construct_Dmat(self):
        D = np.zeros((self.m + len(self.inductors), self.m + len(self.inductors)))
        for index, ind in enumerate(self.inductors):
            D[index + self.m][index + self.m] -= (ind.value / self.time_step) 
        
        return D
    
    def construct_Imat(self):
        i = np.zeros((self.n, 1))
        for cs in self.current_sources:
            if cs.node1 != 0:
                i[cs.node1 - 1][0] += cs.value 
            if cs.node2 != 0:
                i[cs.node2 - 1][0] -= cs.value
        
        # Applying capacitors stamp for backward euler
        for cap in self.capacitors:
            if cap.node1 != 0:
                i[cap.node1 - 1][0] += ( (cap.value/self.time_step) * cap.initial_value )
            if cap.node2 != 0:
                i[cap.node2 - 1][0] -= ( (cap.value/self.time_step) * cap.initial_value )
        return i
    
    def construct_Emat(self):
        e = np.zeros((self.m + len(self.inductors), 1))
        for index, voltage_source in enumerate(self.voltage_sources):
            e[index][0] = voltage_source.value
        
        for index, ind in enumerate(self.inductors):
            e[index + self.m][0] -= ((ind.value/self.time_step) * ind.initial_value)
        
        return e
    
    def update_Imat(self, z_prev):
        i = np.zeros((self.n, 1))
        for cs in self.current_sources:
            if cs.node1 != 0:
                i[cs.node1 - 1][0] += cs.value 
            if cs.node2 != 0:
                i[cs.node2 - 1][0] -= cs.value
        
        # Applying capacitors stamp for backward euler
        for cap in self.capacitors:
            if cap.node1 != 0:
                i[cap.node1 - 1][0] += ( (cap.value/self.time_step) * z_prev[cap.node1 - 1][0] )
            if cap.node2 != 0:
                i[cap.node2 - 1][0] -= ( (cap.value/self.time_step) * z_prev[cap.node2 - 1][0] )
        return i
    
    def update_Emat(self, z_prev):
        e = np.zeros((self.m + len(self.inductors), 1))
        for index, voltage_source in enumerate(self.voltage_sources):
            e[index][0] = voltage_source.value
        
        for index, ind in enumerate(self.inductors):
            e[index + self.m][0] -= ((ind.value/self.time_step) * z_prev[index + self.m][0])
        
        return e
    
    def construct_Zmat(self, i_mat, e_mat):
        return np.concatenate((i_mat,e_mat), axis=0)
    
    def construct_Amat(self, G, B, C, D):
        G_C = np.concatenate((G, C), axis=0)
        B_D = np.concatenate((B, D), axis=0)
        return np.concatenate((G_C, B_D), axis=1)
    
    def solve(self):
        G = self.construct_Gmat()
        B = self.construct_Bmat()
        C = self.construct_Cmat(B_matrix=B)
        D = self.construct_Dmat()
        
        i = self.construct_Imat()
        e = self.construct_Emat()
        Z = self.construct_Zmat(i_mat=i, e_mat=e)
        A = self.construct_Amat(G=G, B=B, C=C, D=D)
        inverse_A = np.linalg.inv(A)
        
        for x in range (0,self.num_iter):
            X = np.matmul(inverse_A, Z)
            print(X)
            i = self.update_Imat(z_prev=X)
            e = self.update_Emat(X)
            Z = self.construct_Zmat(i_mat=i, e_mat=e)
        return X

In [141]:
def read_file(filePath):
    file = open(filePath, 'r')
    lines = file.readlines()
    
    time_step = float(lines[0])
    num_iter = float(lines[1])
    nodes = set()
    
    circuit_components = defaultdict(list)
    
    for line in lines[2:-1]:
        name, node1, node2, value, initial_value = line.strip('\n').split(' ')
        nodes.add(node1)
        nodes.add(node2)
        
        component = Component(name, int(node1[-1]), int(node2[-1]), float(value), float(initial_value))
        circuit_components[name].append(component)
        
    return time_step, num_iter, circuit_components, len(nodes) - 1, len(circuit_components['Vsrc'])

In [142]:
time_step, num_iter, components, n, m = read_file('./testcases/8.txt')

In [143]:
simulation = Simulation(time_step=time_step, num_iter=int(num_iter), components=components, m=m, n=n)
simulation.solve()

[[0.08196721]
 [0.98360656]
 [0.09836066]]
[[0.14915345]
 [0.97016931]
 [0.17898414]]
[[0.20422414]
 [0.95915517]
 [0.24506897]]
[[0.24936405]
 [0.95012719]
 [0.29923686]]
[[0.28636398]
 [0.9427272 ]
 [0.34363677]]
[[0.31669178]
 [0.93666164]
 [0.38003014]]
[[0.34155064]
 [0.93168987]
 [0.40986077]]
[[0.36192676]
 [0.92761465]
 [0.43431211]]
[[0.37862849]
 [0.9242743 ]
 [0.45435419]]
[[0.39231843]
 [0.92153631]
 [0.47078212]]
[[0.4035397 ]
 [0.91929206]
 [0.48424764]]
[[0.41273746]
 [0.91745251]
 [0.49528495]]
[[0.42027661]
 [0.91594468]
 [0.50433193]]
[[0.42645623]
 [0.91470875]
 [0.51174748]]
[[0.4315215]
 [0.9136957]
 [0.5178258]]
[[0.43567336]
 [0.91286533]
 [0.52280804]]
[[0.43907653]
 [0.91218469]
 [0.52689183]]
[[0.44186601]
 [0.9116268 ]
 [0.53023921]]
[[0.44415246]
 [0.91116951]
 [0.53298296]]
[[0.44602661]
 [0.91079468]
 [0.53523193]]


array([[0.44602661],
       [0.91079468],
       [0.53523193]])